In [85]:
import numpy as np
import scipy.special as scisp
import scipy.integrate as integ
import mpmath
from scipy import constants as const
import Green_functions as gr
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import scipy as sc
import nanonis 
%matplotlib qt

# Tinkham

# Definitions T=0

In [99]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta**2-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(Delta,t)*(1-fermi(T,t)),bcs(Delta,x)*fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs(Delta,x)*fermi(T,x))/R2


def p(V,R1,R2,C1,C2,Q0,Delta,T):
    m1=(Gamma1(V,R1,C1,C2,0,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-1,Q0,Delta,T))
    e1=(Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,0,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T))
    e2=(Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,1,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T))
    m2=(Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-2,Q0,Delta,T))
    e3=(Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,2,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,3,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T))
    m3=(Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-3,Q0,Delta,T))
    p0=np.divide(1,1+m1+e1+m1*m2+e1*e2+m1*m2*m3+e1*e2*e3)
    p1=np.multiply(p0,e1)
    p_1=np.multiply(p0,m1)
    p2=np.multiply(p0,e1*e2)
    p_2=np.multiply(p0,m1*m2)
    p3=np.multiply(p0,e1*e2*e3)
    p_3=np.multiply(p0,m1*m2*m3)
    return p0,p_1,p1,p_2,p2,p_3,p3

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma1(V,R1,C1,C2,0,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)
    G1=Gamma1(V,R1,C1,C2,1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)
    G2=Gamma1(V,R1,C1,C2,2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)
    G3=Gamma1(V,R1,C1,C2,3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma1(V,R1,C1,C2,-3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)

def current2(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma2(V,R2,C1,C2,0,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T)
    G1=Gamma2(V,R2,C1,C2,1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma2(V,R2,C1,C2,-1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T)
    G2=Gamma2(V,R2,C1,C2,2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma2(V,R2,C1,C2,-2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T)
    G3=Gamma2(V,R2,C1,C2,3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma2(V,R2,C1,C2,-3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,3,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)


In [187]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta**2-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( (1-fermi(T,t)),fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),fermi(T,x))/R2

def conditions(V,C1,C2,n,Q0,Delta):
    if V+(n+Q0)/C2-(C1+C2)/(2*C1*C2)-2*Delta*(C1+C2)/C2>0:
        a=1
    if V+(n+Q0)/C2-(C1+C2)/(2*C1*C2)-2*Delta*(C1+C2)/C2>0:
        a=0
    if V+(n+Q0)/C2+(C1+C2)/(2*C1*C2)+2*Delta*(C1+C2)/C2<0:
        b=1
    if V+(n+Q0)/C2+(C1+C2)/(2*C1*C2)+2*Delta*(C1+C2)/C2>0:
        b=0
    if V-(n+Q0)/C1-(C1+C2)/(2*C1*C2)-Delta*(C1+C2)/C1>0:
        c=1
    if V-(n+Q0)/C1-(C1+C2)/(2*C1*C2)-Delta*(C1+C2)/C1<0:
        c=0
    if V-(n+Q0)/C1+(C1+C2)/(2*C1*C2)+Delta*(C1+C2)/C1<0:
        d=1
    if V-(n+Q0)/C1+(C1+C2)/(2*C1*C2)+Delta*(C1+C2)/C1>0:
        d=0
    else:
        a=0
        b=0
        c=0
        d=0
    return [a,b,c,d]

def cond(V,R1,R2,C1,C2,n,Q0,Delta,T):
    if Gamma1(V,R1,C1,C2,n,Q0,Delta,T)>0.01:
        a=1
    if Gamma1(V,R1,C1,C2,n,Q0,Delta,T)<0.01:
        a=0
    if Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)>0.01:
        b=1
    if Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)<0.01:
        b=0
    if Gamma2(V,R2,C1,C2,n,Q0,Delta,T)>0.01:
        c=1
    if Gamma2(V,R2,C1,C2,n,Q0,Delta,T)<0.01:
        c=0
    if Gamma2(-V,R2,C1,C2,-n,-Q0,Delta,T)>0.01:
        d=1
    if Gamma2(-V,R2,C1,C2,-n,-Q0,Delta,T)<0.01:
        d=0
    return [a,b,c,d]

def p(V,R1,R2,C1,C2,Q0,Delta,T):
    m1=(Gamma1(V,R1,C1,C2,0,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-1,Q0,Delta,T))
    e1=(Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,0,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T))
    e2=(Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,1,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T))
    m2=(Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-2,Q0,Delta,T))
    e3=(Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,2,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,3,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T))
    m3=(Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-3,Q0,Delta,T))
    p0=np.divide(1,1+m1+e1+m1*m2+e1*e2+m1*m2*m3+e1*e2*e3)
    p1=np.multiply(p0,e1)
    p_1=np.multiply(p0,m1)
    p2=np.multiply(p0,e1*e2)
    p_2=np.multiply(p0,m1*m2)
    p3=np.multiply(p0,e1*e2*e3)
    p_3=np.multiply(p0,m1*m2*m3)
    return p0,p_1,p1,p_2,p2,p_3,p3

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma1(V,R1,C1,C2,0,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)
    G1=Gamma1(V,R1,C1,C2,1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)
    G2=Gamma1(V,R1,C1,C2,2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)
    G3=Gamma1(V,R1,C1,C2,3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma1(V,R1,C1,C2,-3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)

def current2(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma2(V,R2,C1,C2,0,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T)
    G1=Gamma2(V,R2,C1,C2,1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma2(V,R2,C1,C2,-1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T)
    G2=Gamma2(V,R2,C1,C2,2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma2(V,R2,C1,C2,-2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T)
    G3=Gamma2(V,R2,C1,C2,3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma2(V,R2,C1,C2,-3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,3,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)


# P4

In [143]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(Delta,t)*(1-fermi(T,t)),bcs(Delta,x)*fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs(Delta,x)*fermi(T,x))/R2



def p(V,R1,R2,C1,C2,Q0,Delta,T):
    m1=(Gamma1(V,R1,C1,C2,0,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-1,Q0,Delta,T))
    e1=(Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,0,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T))
    e2=(Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,1,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T))
    m2=(Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-2,Q0,Delta,T))
    e3=(Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,2,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,3,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T))
    m3=(Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-3,Q0,Delta,T))
    e4=(Gamma1(-V,R1,C1,C2,-3,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,3,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,4,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-4,-Q0,Delta,T))
    m4=(Gamma1(V,R1,C1,C2,-3,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,3,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,4,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-4,Q0,Delta,T))
    p0=np.divide(1,1+m1+e1+m1*m2+e1*e2+m1*m2*m3+e1*e2*e3+m1*m2*m3*m4+e1*e2*e3*e4)
    p1=np.multiply(p0,e1)
    p_1=np.multiply(p0,m1)
    p2=np.multiply(p0,e1*e2)
    p_2=np.multiply(p0,m1*m2)
    p3=np.multiply(p0,e1*e2*e3)
    p_3=np.multiply(p0,m1*m2*m3)
    p4=np.multiply(p0,e1*e2*e3*e4)
    p_4=np.multiply(p0,m1*m2*m3*m4)
    return p0,p_1,p1,p_2,p2,p_3,p3,p_4,p4

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3,p_4,p4=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma1(V,R1,C1,C2,0,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)
    G1=Gamma1(V,R1,C1,C2,1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)
    G2=Gamma1(V,R1,C1,C2,2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)
    G3=Gamma1(V,R1,C1,C2,3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma1(V,R1,C1,C2,-3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)
    G4=Gamma1(V,R1,C1,C2,4,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-4,-Q0,Delta,T)
    G_4=Gamma1(V,R1,C1,C2,-4,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,4,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)+np.multiply(p4,G4)+np.multiply(p_4,G_4)

def current2(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3,p_4,p4=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma2(V,R2,C1,C2,0,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T)
    G1=Gamma2(V,R2,C1,C2,1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma2(V,R2,C1,C2,-1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T)
    G2=Gamma2(V,R2,C1,C2,2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma2(V,R2,C1,C2,-2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T)
    G3=Gamma2(V,R2,C1,C2,3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma2(V,R2,C1,C2,-3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,3,-Q0,Delta,T)
    G4=Gamma2(V,R2,C1,C2,4,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-4,-Q0,Delta,T)
    G_4=Gamma2(V,R2,C1,C2,-4,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,4,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)+np.multiply(p4,G4)+np.multiply(p_4,G_4)


In [95]:
V=np.linspace(-6,6,300)

file='C:/Users/jonor/Desktop/PhD/Lanak/Coulomb gap/S150722_025.dat'

sp=nanonis.biasSpectroscopy()

sp.load(file)
 
f,ax = plt.subplots(1)

f.subplots_adjust(bottom=0.6)

 

ax1 = f.add_axes([0.10, 0.10, 0.65, 0.03])
ax2 = f.add_axes([0.10, 0.15, 0.65, 0.03])
ax3 = f.add_axes([0.10, 0.20, 0.65, 0.03])
ax4 = f.add_axes([0.10, 0.25, 0.65, 0.03])
ax5 = f.add_axes([0.10, 0.30, 0.65, 0.03])
ax6 = f.add_axes([0.10, 0.35, 0.65, 0.03])
ax7 = f.add_axes([0.10, 0.40, 0.65, 0.03])


ax1_s = plt.Slider(ax1,'R1',1,1000000,valinit=1)
ax2_s = plt.Slider(ax2,'R2',0,10,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0,10000,valinit=1)
ax4_s = plt.Slider(ax4,'C2',0,10000,valinit=1)
ax5_s = plt.Slider(ax5,'Q0',-1.5,1.5,valinit=0)
ax6_s = plt.Slider(ax6,'Delta',0.0,1.5,valinit=1)
ax7_s = plt.Slider(ax7,'T',0.005,0.1,valinit=0.08)



def update(val):
    ax.clear()
    a=np.gradient(current(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val))
    b=np.gradient(current2(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val))
    ax.plot(V,a/a[0])
    ax.plot(V,b/b[0])
    #ax.plot(sp.bias*700,sp.conductance/sp.conductance[0])



ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)

0

In [144]:
V=np.linspace(-7,7,200)

f,ax = plt.subplots(1)

f.subplots_adjust(bottom=0.6)

 

ax1 = f.add_axes([0.10, 0.10, 0.65, 0.03])

ax2 = f.add_axes([0.10, 0.15, 0.65, 0.03])

ax3 = f.add_axes([0.10, 0.20, 0.65, 0.03])

ax4 = f.add_axes([0.10, 0.25, 0.65, 0.03])

ax5 = f.add_axes([0.10, 0.30, 0.65, 0.03])

ax6 = f.add_axes([0.10, 0.35, 0.65, 0.03])

ax7 = f.add_axes([0.10, 0.40, 0.65, 0.03])




ax1_s = plt.Slider(ax1,'R1',0,10000,valinit=1)

ax2_s = plt.Slider(ax2,'R2',0,10,valinit=1)

ax3_s = plt.Slider(ax3,'C1',0,1,valinit=1)

ax4_s = plt.Slider(ax4,'C2',0,1,valinit=1)

ax5_s = plt.Slider(ax5,'Q0',-1,1,valinit=0)

ax6_s = plt.Slider(ax6,'Delta',0.5,1.5,valinit=1)

ax7_s = plt.Slider(ax7,'T',0.005,1,valinit=0.01)




def update(val):

    ax.clear()

    # ax.plot(V,np.gradient(current(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val)))

    # ax.plot(V,np.gradient(current2(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val)))

    ps=p(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val)

    labels = [0,-1,1,-2,2,-3,3,-4,4]

    n=0

    for i in ps:

        ax.plot(V,i,label=str(labels[n]))

        n+=1

    ax.legend()






ax1_s.on_changed(update)

ax2_s.on_changed(update)

ax3_s.on_changed(update)

ax4_s.on_changed(update)

ax5_s.on_changed(update)

ax6_s.on_changed(update)

ax7_s.on_changed(update)

0

C:\Users\jonor\AppData\Local\Temp/ipykernel_13564/465675385.py:6: RuntimeWarning: overflow encountered in exp
  return np.divide(1,1+np.exp(x/T))


In [3]:
def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-15*Delta,15*Delta,4000)
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot((1-fermi(T,t)),fermi(T,x) )/R1
def bcs(delta,x):
    eta=delta/50
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta**2-(x+eta*1j)**2)))
def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)
V=np.linspace(-7,7,1000)
C1=100
C2=100
R2=1
Q0=0
T=0.01
Delta=1
Ec=(C1+C2)/(2*C1*C2)
k=C2/(C1+C2)
plt.plot(V,np.gradient(Gamma1(V,R2,C1,C2,0,Q0,Delta,T)-Gamma1(-V,R2,C1,C2,0,-Q0,Delta,T)))
plt.axvline(Ec)

C:\Users\jonor\AppData\Local\Temp/ipykernel_20328/465675385.py:6: RuntimeWarning: overflow encountered in exp
  return np.divide(1,1+np.exp(x/T))


# GENERAL P MODEL

In [23]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C2/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(Delta,t)*(1-fermi(T,t)),bcs(Delta,x)*fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-10*Delta,10*Delta,2000)
    k=C1/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot((1-fermi(T,t)),bcs(Delta,x)*fermi(T,x))/R2


def PN(V,R1,R2,C1,C2,Q0,Delta,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,n,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,Delta,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,n,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,Delta,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,Delta,T):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,Delta,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,Delta,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,Delta,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)

def G2n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,Delta,T)

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    return I


def current2(V,R1,R2,C1,C2,Q0,Delta,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    return I


2 delta

In [110]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-8,8,1500)
    k=C2/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(1.0,t)*(1-fermi(T,t)),bcs(Delta,x)*fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-8,8,1500)
    k=C1/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs(Delta,x)*fermi(T,x))/R2


def PN(V,R1,R2,C1,C2,Q0,Delta,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,n,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,Delta,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,n,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,Delta,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,Delta,T):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,Delta,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,Delta,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,Delta,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)

def G2n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,Delta,T)

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    return I


def current2(V,R1,R2,C1,C2,Q0,Delta,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    return I


3 Delta

In [36]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-8,8,1500)
    k=C2/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(Delta,t)*(1-fermi(T,t)),bcs(1.0,x)*fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,Delta2,T):
    x=np.linspace(-8,8,1500)
    k=C1/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(bcs(Delta2,t)*(1-fermi(T,t)),bcs(Delta,x)*fermi(T,x))/R2


def PN(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,n,-Q0,Delta,Delta2,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,Delta,Delta2,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,n,Q0,Delta,Delta2,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,Delta,Delta2,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,Delta,Delta2,T):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,Delta,Delta2,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)

def G2n(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,Delta,Delta2,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,Delta,Delta2,T)

def current(V,R1,R2,C1,C2,Q0,Delta,Delta2,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,Delta2,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    return I


def current2(V,R1,R2,C1,C2,Q0,Delta,Delta2,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,Delta2,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,Delta2,T,-i)
    return I


In [3]:
V=np.linspace(-7,7,400)

a=np.gradient(current(V,1,10000,0.55,0.01,-0.13,1.7,0.2))
spec=nanonis.biasSpectroscopy()
spec.load('C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/Magnetic field Coulomb/S20mT230405_007.dat')

plt.plot(V,a/a[0])
plt.plot(np.array(spec.bias)*1000,np.array(spec.conductance)/np.array(spec.conductance)[0])

In [22]:
V=np.linspace(-6,6,200)

file='C:/Users/jonor/Desktop/PhD/Lanak/Coulomb gap/S150722_025.dat'

sp=nanonis.biasSpectroscopy()

sp.load(file)
 
f,ax = plt.subplots(1)

f.subplots_adjust(bottom=0.6)

 

ax1 = f.add_axes([0.10, 0.10, 0.65, 0.03])
ax2 = f.add_axes([0.10, 0.15, 0.65, 0.03])
ax3 = f.add_axes([0.10, 0.20, 0.65, 0.03])
ax4 = f.add_axes([0.10, 0.25, 0.65, 0.03])
ax5 = f.add_axes([0.10, 0.30, 0.65, 0.03])
ax6 = f.add_axes([0.10, 0.35, 0.65, 0.03])
ax7 = f.add_axes([0.10, 0.40, 0.65, 0.03])


ax1_s = plt.Slider(ax1,'R1',1,500,valinit=1)
ax2_s = plt.Slider(ax2,'R2',0,1,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0,10,valinit=1)
ax4_s = plt.Slider(ax4,'C2',0,10,valinit=1)
ax5_s = plt.Slider(ax5,'Q0',-1.5,1.5,valinit=0)
ax6_s = plt.Slider(ax6,'Delta',0.0,1.5,valinit=1)
ax7_s = plt.Slider(ax7,'T',0.01,0.5,valinit=0.1)



def update(val):
    ax.clear()
    a=np.gradient(current(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val))
    b=np.gradient(current2(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val))
    ax.plot(V,a/a[0])
    ax.plot(V,b/b[0])
    ax.plot(sp.bias*1000,sp.conductance/sp.conductance[0])



ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)
ax7_s.on_changed(update)


0

3 Delta

In [43]:
V=np.linspace(-6,6,200)

file='C:/Users/jonor/Desktop/PhD/Lanak/Coulomb gap/S150722_025.dat'

sp=nanonis.biasSpectroscopy()

sp.load(file)
 
f,ax = plt.subplots(1)

f.subplots_adjust(bottom=0.6)

 

ax1 = f.add_axes([0.10, 0.10, 0.65, 0.03])
ax2 = f.add_axes([0.10, 0.15, 0.65, 0.03])
ax3 = f.add_axes([0.10, 0.20, 0.65, 0.03])
ax4 = f.add_axes([0.10, 0.25, 0.65, 0.03])
ax5 = f.add_axes([0.10, 0.30, 0.65, 0.03])
ax6 = f.add_axes([0.10, 0.35, 0.65, 0.03])
ax7 = f.add_axes([0.10, 0.40, 0.65, 0.03])
ax8 = f.add_axes([0.10, 0.45, 0.65, 0.03])


ax1_s = plt.Slider(ax1,'R1',1,500,valinit=1)
ax2_s = plt.Slider(ax2,'R2',0,1,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0,10,valinit=1)
ax4_s = plt.Slider(ax4,'C2',0,10,valinit=1)
ax5_s = plt.Slider(ax5,'Q0',-1.5,1.5,valinit=0)
ax6_s = plt.Slider(ax6,'Delta',0.0,1.5,valinit=1)
ax7_s = plt.Slider(ax7,'Delta2',0.0,1.5,valinit=1)
ax8_s = plt.Slider(ax8,'T',0.01,0.5,valinit=0.1)

def update(val):
    ax.clear()
    a=np.gradient(current(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val,ax8_s.val))
    b=np.gradient(current2(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val,ax7_s.val,ax8_s.val))

    ax.plot(V,a/a[0])
    ax.plot(V,b/b[0])
    #ax.plot(sp.bias*1000,sp.conductance/sp.conductance[0])


ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)
ax7_s.on_changed(update)
ax8_s.on_changed(update)


0

In [40]:
plt.plot(V,Gamma1(V,1,1,1,0,0,1,0.1))
plt.plot(V,Gamma2(V,1,1,1,0,0,1,1,0.1))


# Any pn normal metal

In [107]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    x=np.linspace(-20,20,3000)
    k=C2/(C1+C2)
    Ec=(C1*C2)/((C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot((1-fermi(T,t)),fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    x=np.linspace(-20,20,3000)
    k=C1/(C1+C2)
    Ec=(C1*C2)/((C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),fermi(T,x))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.0001 and np.sum(p_n)<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    return I


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    return I

In [113]:
V=np.linspace(-10,10,200)
C1=0.4
C2=0.1
Q0=0.3
R1=1
R2=10000
#y=all_p(V,R1,R2,C1,C2,Q0,0.01,4)
#plt.plot(V,y[1][0])
#plt.plot(V,y[1][1])
#plt.plot(V,y[1][2])
#plt.plot(V,y[2][0])
#plt.plot(V,y[2][1])
#plt.plot(V,y[2][2])
#plt.plot(V,y[0])
h=0

y=np.gradient(current(V,R1,R2,C1,C2,Q0,0.7))
plt.plot(V,y/y[0]+h)
yy=[]
for i in V:
        yy.append(currentT(i,C1,C2,-Q0,T))
plt.plot(V,np.gradient(yy)/np.gradient(yy)[0]+h)


C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:14: RuntimeWarning: overflow encountered in exp
  return -E2p(V,C1,C2,Q0,n)/(1-np.exp(E2p(V,C1,C2,Q0,n)/T))
C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:17: RuntimeWarning: overflow encountered in exp
  return -E2m(V,C1,C2,Q0,n)/(1-np.exp(E2m(V,C1,C2,Q0,n)/T))


In [44]:
V=np.linspace(-5,5,100)

 
f,ax = plt.subplots(1)

f.subplots_adjust(bottom=0.6)

 

ax1 = f.add_axes([0.10, 0.10, 0.65, 0.03])
ax2 = f.add_axes([0.10, 0.15, 0.65, 0.03])
ax3 = f.add_axes([0.10, 0.20, 0.65, 0.03])
ax4 = f.add_axes([0.10, 0.25, 0.65, 0.03])
ax5 = f.add_axes([0.10, 0.30, 0.65, 0.03])
ax6 = f.add_axes([0.10, 0.40, 0.65, 0.03])


ax1_s = plt.Slider(ax1,'R1',1,1000,valinit=1)
ax2_s = plt.Slider(ax2,'R2',0,10,valinit=1)
ax3_s = plt.Slider(ax3,'C1',0,5,valinit=1)
ax4_s = plt.Slider(ax4,'C2',0,5,valinit=1)
ax5_s = plt.Slider(ax5,'Q0',-0.5,0.5,valinit=0)
ax6_s = plt.Slider(ax6,'T',0.005,0.1,valinit=0.01)



def update(val):
    ax.clear()
    a=np.gradient(current(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val))
    b=np.gradient(current2(V,ax1_s.val,ax2_s.val,ax3_s.val,ax4_s.val,ax5_s.val,ax6_s.val))
    ax.plot(V,a/a[0])
    ax.plot(V,b/b[0])




ax1_s.on_changed(update)
ax2_s.on_changed(update)
ax3_s.on_changed(update)
ax4_s.on_changed(update)
ax5_s.on_changed(update)
ax6_s.on_changed(update)


0

# Tinkham Metal

In [88]:
def E2p(V,C1,C2,Q0,n):
    C=(C1+C2)
    return (0.5+(n-Q0)-C1*V)/C
def E2m(V,C1,C2,Q0,n):
    C=(C1+C2)
    return (0.5-(n-Q0)+C1*V)/C

def N(V,C2,Q0):
    b=(-C2*V+Q0+0.5)
    return b-np.mod(b,1)

def Gamma2p(V,C1,C2,Q0,T):
    n=N(V,C2,Q0)
    return -E2p(V,C1,C2,Q0,n)/(1-np.exp(E2p(V,C1,C2,Q0,n)/T))
def Gamma2m(V,C1,C2,Q0,T):
    n=N(V,C2,Q0)
    return -E2m(V,C1,C2,Q0,n)/(1-np.exp(E2m(V,C1,C2,Q0,n)/T))
def currentT(V,C1,C2,Q0,T):
    return Gamma2p(V,C1,C2,Q0,T)-Gamma2m(V,C1,C2,Q0,T)

C1=2
C2=0.1
Q0=np.linspace(-0.49,0.49,10)
T=0.01
V=np.linspace(-10,10,500)
h=0
for j in Q0:
    y=[]
    for i in V:
        y.append(currentT(i,C1,C2,j,T))
    plt.plot(V,np.gradient(y)/np.gradient(y)[0]+h)
    h+=1


C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:14: RuntimeWarning: overflow encountered in exp
  return -E2p(V,C1,C2,Q0,n)/(1-np.exp(E2p(V,C1,C2,Q0,n)/T))
C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:17: RuntimeWarning: overflow encountered in exp
  return -E2m(V,C1,C2,Q0,n)/(1-np.exp(E2m(V,C1,C2,Q0,n)/T))
C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:14: RuntimeWarning: overflow encountered in exp
  return -E2p(V,C1,C2,Q0,n)/(1-np.exp(E2p(V,C1,C2,Q0,n)/T))
C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:17: RuntimeWarning: overflow encountered in exp
  return -E2m(V,C1,C2,Q0,n)/(1-np.exp(E2m(V,C1,C2,Q0,n)/T))
C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:14: RuntimeWarning: overflow encountered in exp
  return -E2p(V,C1,C2,Q0,n)/(1-np.exp(E2p(V,C1,C2,Q0,n)/T))
C:\Users\jonor\AppData\Local\Temp/ipykernel_36396/2382579946.py:17: RuntimeWarning: overflow encountered in exp
  return -E2m(V,C1,C2,Q0,n)/

# PE One junction

In [389]:
def ReZ(x,R,C):
    return R/(1+(x*R*C)**2)

def fermi(x,T):
    return 1/(1+np.exp(x/T))

def bcs(x,delta):
    eta=delta/500
    return np.imag((x+1j*eta)/np.sqrt(delta**2-(x+1j*eta)**2))

def f(t,x,R,C,T):
    return np.trapz(2*(ReZ(x,R,C)/x)*( (1/np.tanh(x/(2*T)))*(np.cos(x*t)-1)- 1j*np.sin(x*t)  ),x)

def P(R,C,T):
    dis=1/1
    N=4000
    t=np.linspace(-2*dis/2,2*dis/2,N)
    x=np.linspace(0.01,100,5000)
    freq=np.fft.fftfreq(N,d=dis)
    a=np.argsort(freq)
    y=[]
    for i in t:
        y.append(np.exp(f(i,x,R,C,T)))
    y=np.real(np.fft.fft(y))
    t=[]
    for i in a:
        t.append(y[i])
    return np.sort(freq),t

delta=0.05
T=0.01
y=P(1,0.01,1)
x=y[0]
p=np.abs(y[1])
#Q0=0
E=np.linspace(-5*delta,5*delta,4000)
V=np.linspace(-3*delta,3*delta,100)
a,b=np.meshgrid(x,E)
E2=a-b
#c,d=np.meshgrid(E,V)
a,b=np.meshgrid(E,V)
#E3p=a-b
#a,b=np.meshgrid(E,V-Q0)
#E3m=a+b


#plt.plot(x,p/np.max(p))

#current=np.dot(np.dot(bcs(E3p,delta)*fermi(E3p,T),bcs(E2,delta)*fermi(E2,T)),p)-np.dot(np.dot(bcs(E3m,delta)*fermi(E3m,T),bcs(E2,delta)*fermi(E2,T)),p)
#y=np.gradient(current)
#plt.plot(V,y/y[0])
#
#current=np.dot(bcs(c-d,delta)*fermi(c-d,T),bcs(-E,delta)*fermi(-E,T))-np.dot(bcs(c+d,delta)*fermi(c+d,T),bcs(-E,delta)*fermi(-E,T))
#y=np.gradient(current)
#plt.plot(V,y/y[0])
#plt.legend(['C','T'])

# PE two junctions

In [386]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def PE1(x,C1,C2,T):
    k=C2/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a=4*k**2*Ec*T
    return (1/np.sqrt(a*np.pi))*np.exp(-(x-k**2*Ec)**2/a)

def PE2(x,C1,C2,T):
    k=C1/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a=4*k**2*Ec*T
    return (1/np.sqrt(a*np.pi))*np.exp(-(x-k**2*Ec)**2/a)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    x=np.linspace(-20,20,3000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a-b
    return np.dot(PE1(t,C1,C2,T),x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    x=np.linspace(-20,20,3000)
    a,b=np.meshgrid(x,E2(C1,C2,V,n,Q0))
    t=a-b
    return np.dot(PE2(t,C1,C2,T),x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.01 and p_n<0.01:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    return I


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    return I


In [387]:
V=np.linspace(-4,4,100)
C1=0.1
C2=1
Q0=0.1
R1=10000
R2=1

#y=all_p(V,R1,R2,C1,C2,Q0,0.01,4)
#plt.plot(V,y[1][0])
#plt.plot(V,y[1][1])
#plt.plot(V,y[1][2])
#plt.plot(V,y[2][0])
#plt.plot(V,y[2][1])
#plt.plot(V,y[2][2])
#plt.plot(V,y[0])
plt.plot(V,np.gradient(current(V,R1,R2,C1,C2,Q0,0.01)))

C:\Users\jonor\AppData\Local\Temp/ipykernel_10592/2551971085.py:28: RuntimeWarning: overflow encountered in exp
  return np.dot(PE1(t,C1,C2,T),x/(1-np.exp(-x/T)))/R1
C:\Users\jonor\AppData\Local\Temp/ipykernel_10592/2551971085.py:34: RuntimeWarning: overflow encountered in exp
  return np.dot(PE2(t,C1,C2,T),x/(1-np.exp(-x/T)))/R2
C:\Users\jonor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\jonor\AppData\Local\Temp/ipykernel_10592/2551971085.py:59: RuntimeWarning: invalid value encountered in double_scalars
  pn=p0*np.prod(a)
C:\Users\jonor\AppData\Local\Temp/ipykernel_10592/2551971085.py:39: RuntimeWarning: divide by zero encountered in true_divide
  en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))

KeyboardInterrupt: 

# Josephson Current

In [20]:
def bcs(delta,x):
    eta=delta/200
    return np.imag(np.divide(x+eta*1j,np.sqrt(delta-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(2*n+2*Q0-1)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def PE1(x,C1,C2,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*(C1*C2))
    a=4*k**2*Ec*T
    return (1/np.sqrt(a*np.pi))*np.exp(-(x-k**2*Ec)**2/a)

def PE2(x,C1,C2,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*(C1*C2))
    a=4*k**2*Ec*T
    return (1/np.sqrt(a*np.pi))*np.exp(-(x-k**2*Ec)**2/a)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    x=E1(C1,C2,2*V,n,Q0)
    return [(PE1(x,C1,C2,T)-PE1(x,C1,C2,T))/R1]

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-8,8,3000)
    k=C1/(C1+C2)
    Ec=(1)/(2*(C1+C2))
    a,b=np.meshgrid(x,-k**2*Ec+E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs(Delta,x)*fermi(T,x))/R2

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    p10=Gamma1(V,R1,C1,C2,0,Q0,T)[0]
    m10=Gamma1(-V,R1,C1,C2,0,-Q0,T)[0]
    p20=Gamma2(V,R2,C1,C2,0,Q0,Delta,T)[0]
    m20=Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T)[0]
    p11=Gamma1(V,R1,C1,C2,1,Q0,T)[0]
    m11=Gamma1(-V,R1,C1,C2,-1,-Q0,T)[0]
    p21=Gamma2(V,R2,C1,C2,1,Q0,Delta,T)[0]
    m21=Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T)[0]
    p12=Gamma1(V,R1,C1,C2,2,Q0,T)[0]
    m12=Gamma1(-V,R1,C1,C2,-2,-Q0,T)[0]
    p22=Gamma2(V,R2,C1,C2,2,Q0,Delta,T)[0]
    m22=Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T)[0]
    m2_1=Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T)[0]
    m1_2=Gamma1(-V,R1,C1,C2,2,-Q0,T)[0]
    m1_1=Gamma1(-V,R1,C1,C2,1,-Q0,T)[0]
    p1_1=Gamma1(V,R1,C1,C2,-1,Q0,T)[0]
    p2_1=Gamma2(V,R2,C1,C2,-1,Q0,Delta,T)[0]
    m2_2=Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T)[0]
    p1_2=Gamma1(V,R1,C1,C2,-2,Q0,T)[0]
    matrix=np.array([[p10+m10+p20+m20,-p21,-p12,-m2_1,-m1_2],[-m20,p11+m11+p21+m21,-p22,-m1_1,0],[-m10,-m21,p12+m12+p22+m22,0,0],[-p20,-p11,0,p1_1+m1_1+p2_1+m2_1,-m2_2],[1,1,1,1,1]])
    p=np.dot(np.linalg.inv(matrix),[0,0,0,0,1])
    return p[0]*(p10-m10)+p[1]*(p11-m11)+p[2]*(p12-m12)+p[3]*(p1_1-m1_1)+p[4]*(p1_2-m1_2)

Delta=10
R1=10
R2=10
C1=10
C2=10
Q0=0
T=0.1

I=[]
V=np.linspace(-Delta/2,Delta/2,100)
for i in V:
    I.append(current(i,R1,R2,C1,C2,Q0,Delta,T))

plt.plot(V,I)